In [ ]:
# 查看当前挂载的数据集目录
!ls /home/kesci/input/

In [ ]:
# 查看个人持久化工作区文件
!ls /home/kesci/work/

In [ ]:
# 查看当前kernerl下的package
!pip list --format=columns

# 训练

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec




class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = 3
        self.fc_hidden_size = 512
        self.n_class = 1
        self.train_batch_size=512
        self.vaild_batch_size=512
        self.lr=0.0015
        self.decay_step=20000
        self.decay_rate=0.8
        self.hidden_size=200
        self.n_layer_bigru=2
        self.dynamic_embedding_size=30
        self.vocabulary_size = 0


class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        # placeholders
        self.embedding_size=100
        self.initializer=tf.contrib.layers.xavier_initializer()
        self.hidden_size=settings.hidden_size
        self.n_layer_bigru=settings.n_layer_bigru
        self.dynamic_embedding_size=settings.dynamic_embedding_size
        self.vocabulary_size = settings.vocabulary_size

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            self._batch_size = tf.placeholder(tf.int32, [])
            self._keep_prob = tf.placeholder(tf.float32, [])
            self.is_training = tf.placeholder(name='training', dtype=tf.bool)
            


        
        


        
        with tf.variable_scope('bigru_text'):
            output_title = self.bigru_inference(self.embedded_x_query)

        with tf.variable_scope('bigru_content'):
            output_content = self.bigru_inference(self.embedded_x_title)
        

        with tf.variable_scope('fc-bn-layer'):
            self.dnn_all = tf.concat([output_title, output_content], axis=1)
            dnn_bn = self.bn_layer_top(self.dnn_all,'dnn_bn_',self.is_training)

            fn_input_size = int(dnn_bn.shape[-1])
            W_fc = self.weight_variable([fn_input_size, self.fc_hidden_size], name='Weight_fc')
            b_fc = self.bias_variable([self.fc_hidden_size], name='fc_bias')
            fc_out = tf.nn.xw_plus_b(dnn_bn, W_fc, b_fc, name='y_pred')
            fc_relu = tf.nn.relu(fc_out, name="fc_relu")
            fc_bn = self.bn_layer_top(fc_relu,'fc_bn_',self.is_training)
            fc_drop = tf.nn.dropout(fc_bn, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([int(fc_drop.shape[-1]), self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(fc_bn, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred


        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred)


    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step


    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss


    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


        
    def dropout(self, x, keep_prob):
        return tf.nn.dropout(x, keep_prob)
        
    def gru_cell(self):
        with tf.name_scope('gru_cell'):
            cell = rnn.GRUCell(self.hidden_size, reuse=False)
        return rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

    def bi_gru(self, inputs):
        """build the bi-GRU network. 返回个所有层的隐含状态。"""
        cells_fw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        cells_bw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        initial_states_fw = [cell_fw.zero_state(self.batch_size, tf.float32) for cell_fw in cells_fw]
        initial_states_bw = [cell_bw.zero_state(self.batch_size, tf.float32) for cell_bw in cells_bw]
        outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs,
                                                            initial_states_fw=initial_states_fw,
                                                            initial_states_bw=initial_states_bw, dtype=tf.float32)
        return outputs

    def task_specific_attention(self, inputs, output_size,
                                initializer=layers.xavier_initializer(),
                                activation_fn=tf.tanh, scope=None):
        assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
        with tf.variable_scope(scope or 'attention') as scope:
            # u_w, attention 向量
            attention_context_vector = tf.get_variable(name='attention_context_vector', shape=[output_size],
                                                       initializer=initializer, dtype=tf.float32)
            # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
            input_projection = layers.fully_connected(inputs, output_size, activation_fn=activation_fn, scope=scope)
            # 输出 [batch_size, units]
            vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
            attention_weights = tf.nn.softmax(vector_attn, dim=1)
            tf.summary.histogram('attention_weigths', attention_weights)
            weighted_projection = tf.multiply(inputs, attention_weights)
            outputs = tf.reduce_sum(weighted_projection, axis=1)
            return outputs  # 输出 [batch_size, hidden_size*2]


    def bigru_inference(self, X_inputs):
        inputs=X_inputs
        output_bigru = self.bi_gru(inputs)
        output_att = self.task_specific_attention(output_bigru, self.hidden_size*2)
        return output_att

    def bn_layer(self,x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            shape = x.get_shape().as_list()
            gamma = tf.get_variable(scope+"_gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
            beta = tf.get_variable(scope+"_beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
            moving_avg = tf.get_variable(scope+"_moving_mean", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
            moving_var = tf.get_variable(scope+"_moving_variance", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
            if is_training is not None:
                avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
                avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
                var=tf.reshape(var, [var.shape.as_list()[-1]])
                update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
                update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
                control_inputs = [update_moving_avg, update_moving_var]
            else:
                avg = moving_avg
                var = moving_var
                control_inputs = []
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        return output

    def bn_layer_top(self,x, scope, is_training, epsilon=0.001, decay=0.99):
        return tf.cond(
            is_training,
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None),
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True),
        )



def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])
    print("开始加载数据")
    all_data=pd.read_pickle(data_store_path+"/home/kesci/work/second_zzp/090-095.pickle")
    data_split=StratifiedShuffleSplit(n_splits=2,test_size=0.001,random_state=1)
    train_index,vaild_index=data_split.split(all_data['label'],all_data['label']).__next__()
    train_data=all_data.iloc[train_index]
    vaild_data=all_data.iloc[vaild_index]
    print("加载数据完成")
    return train_data,vaild_data

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=True):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

def get_w2v_array(word_list,max_len):
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
      
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
    return array


settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,4)

vaild_batches=np.array(vaild_data)
vaild_batches_y=vaild_batches[:,2]
vaild_batches_x=np.delete(vaild_batches, 2, axis=1)
vaild_data=batch_iter(vaild_batches_x,vaild_batches_y,settings.vaild_batch_size,1000)

print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()
settings.vocabulary_size = len(w2v_model.wv.vocab)
settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)


update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                        settings.decay_rate, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
var_list = tf.trainable_variables()
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
global_step_var=[g for g in g_list if 'Global_Step' in g.name]
var_list += bn_moving_vars
var_list += global_step_var
saver = tf.train.Saver(var_list)
sess.run(tf.global_variables_initializer())
saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/first_zzp/tf_model_bigru/'))

per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
parts_time=[]

from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)


    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    
                             
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
                model.is_training:True,
 
        
    }
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    

        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,
                    model.is_training:False,

        
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
    
        var_list = tf.trainable_variables()
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        global_step_var=[g for g in g_list if 'Global_Step' in g.name]
        var_list += global_step_var
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list)

        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_bigru/rnn", global_step=step)

# 以下是看不到输出了，可以停止kernel，执行下面的代码，接着上面的进度跑

In [ ]:
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)


    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    
                             
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
                model.is_training:True,
 
        
    }
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    

        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,
                    model.is_training:False,

        
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
    
        var_list = tf.trainable_variables()
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        global_step_var=[g for g in g_list if 'Global_Step' in g.name]
        var_list += global_step_var
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list)

        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_bigru/rnn", global_step=step)

In [ ]:
from time import time
for batch in batches:
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)


    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    
                             
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.y_inputs: batch_y,
                 model.batch_size: batch_x.shape[0], 
                 model.keep_prob: 0.8,
                model.is_training:True,
 
        
    }
    _,step,loss_out=sess.run([train_op,model.global_step,model.loss],feed_dict)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print("step {},loss:{:g}".format(step,loss_out))              
    per_train_loss_avg_list.append(loss_out)
    
    if step%(int(50000000/1004))==0:
        print("batch ended")
    if step%500==0:
        time_single_part=sum(parts_time)
        ed_time=(50000000/settings.train_batch_size-step)/500*time_single_part
        print("距离训练完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]
        vaild_x,vaild_y=zip(*(vaild_data.__next__()))
        vaild_y=np.array(vaild_y)[:,None]
        vaild_x=[i.tolist() for i in vaild_x]
        vaild_x=pd.DataFrame(vaild_x)
        vaild_x[[0,1]]=vaild_x[[0,1]].applymap(lambda x:x.split(' '))
        vaild_x=np.array(vaild_x)
    

        vaild_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=vaild_x)
        vaild_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=vaild_x)

        feed_dict = {model.embedded_x_query: vaild_x_query,
                     model.embedded_x_title: vaild_x_title,
                     model.y_inputs: vaild_y,
                     model.batch_size: vaild_x.shape[0], 
                     model.keep_prob: 1,
                    model.is_training:False,

        
        }
    
        step,loss_out=sess.run([model.global_step,model.loss],feed_dict)
        per_vaild_loss_avg_list.append(loss_out)
        per_train_loss_avg=sum(per_train_loss_avg_list)/len(per_train_loss_avg_list)
        print("per_train_loss_avg:",per_train_loss_avg,"**********")
        per_train_loss_avg_list=[]
        print("vaild_loss:",loss_out)
    if step%5000==0:
        per_vaild_loss_avg=sum(per_vaild_loss_avg_list)/len(per_vaild_loss_avg_list)
        per_vaild_loss_avg_list=[]
        print("per_vaild_loss_avg:",per_vaild_loss_avg,"************")
    
        var_list = tf.trainable_variables()
        g_list = tf.global_variables()
        bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
        bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
        global_step_var=[g for g in g_list if 'Global_Step' in g.name]
        var_list += global_step_var
        var_list += bn_moving_vars
        saver = tf.train.Saver(var_list=var_list)

        saver = saver.save(sess, "/home/kesci/work/first_zzp/tf_model_bigru/rnn", global_step=step)

# 预测

In [1]:
# -*- coding:utf-8 -*-

import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  
# os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import tensorflow as tf

from tensorflow.contrib import rnn

import tensorflow.contrib.layers as layers
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit

import tensorflow as tf

import pandas as pd
import numpy as np
from gensim.models import Word2Vec


class Settings(object):

    def __init__(self):


        self.model_name = 'wd_1_2_cnn_max'
        self.query_len = 20
        self.title_len = 20
        self.filter_sizes = 3
        self.fc_hidden_size = 512
        self.n_class = 1
        self.train_batch_size=1024
        self.vaild_batch_size=512
        self.lr=0.0015
        self.decay_step=20000
        self.decay_rate=0.8
        self.hidden_size=200
        self.n_layer_bigru=2
        self.dynamic_embedding_size=30
        self.vocabulary_size = 0


class TEXTCNN(object):

    def __init__(self,  settings):


        self.model_name = settings.model_name
        self.query_len = settings.query_len
        self.title_len = settings.title_len
        self.filter_sizes = settings.filter_sizes
        self.n_class = settings.n_class
        self.fc_hidden_size = settings.fc_hidden_size
        self._global_step = tf.Variable(0, trainable=False, name='Global_Step')
        # placeholders
        self.embedding_size=100
        self.initializer=tf.contrib.layers.xavier_initializer()
        self.hidden_size=settings.hidden_size
        self.n_layer_bigru=settings.n_layer_bigru
        self.dynamic_embedding_size=settings.dynamic_embedding_size
        self.vocabulary_size = settings.vocabulary_size

        with tf.name_scope('Inputs'):
            self.embedded_x_query=tf.placeholder(tf.float32,[None,self.query_len,self.embedding_size])#h r
            self.embedded_x_title=tf.placeholder(tf.float32,[None,self.title_len,self.embedding_size])# r
            self._y_inputs = tf.placeholder(tf.float32, [None, self.n_class], name='y_input')
            self._batch_size = tf.placeholder(tf.int32, [])
            self._keep_prob = tf.placeholder(tf.float32, [])
            self.is_training = tf.placeholder(name='training', dtype=tf.bool)
            


        
        


        
        with tf.variable_scope('bigru_text'):
            output_title = self.bigru_inference(self.embedded_x_query)

        with tf.variable_scope('bigru_content'):
            output_content = self.bigru_inference(self.embedded_x_title)
        

        with tf.variable_scope('fc-bn-layer'):
            self.dnn_all = tf.concat([output_title, output_content], axis=1)
            dnn_bn = self.bn_layer_top(self.dnn_all,'dnn_bn_',self.is_training)

            fn_input_size = int(dnn_bn.shape[-1])
            W_fc = self.weight_variable([fn_input_size, self.fc_hidden_size], name='Weight_fc')
            b_fc = self.bias_variable([self.fc_hidden_size], name='fc_bias')
            fc_out = tf.nn.xw_plus_b(dnn_bn, W_fc, b_fc, name='y_pred')
            fc_relu = tf.nn.relu(fc_out, name="fc_relu")
            fc_bn = self.bn_layer_top(fc_relu,'fc_bn_',self.is_training)
            fc_drop = tf.nn.dropout(fc_bn, self.keep_prob)

        with tf.variable_scope('out_layer'):
            W_out = self.weight_variable([int(fc_drop.shape[-1]), self.n_class], name='Weight_out')
            b_out = self.bias_variable([self.n_class], name='bias_out')
            self._y_pred = tf.nn.xw_plus_b(fc_bn, W_out, b_out, name='y_pred')  # 每个类别的分数 scores
            self._y_pred = self._y_pred


        with tf.name_scope('loss'):
            self._loss = tf.reduce_mean(
                tf.nn.sigmoid_cross_entropy_with_logits(logits=self._y_pred, labels=self._y_inputs))
            self._y_pred = tf.sigmoid(self._y_pred)


    @property
    def keep_prob(self):
        return self._keep_prob

    @property
    def batch_size(self):
        return self._batch_size

    @property
    def global_step(self):
        return self._global_step


    @property
    def y_inputs(self):
        return self._y_inputs

    @property
    def y_pred(self):
        return self._y_pred

    @property
    def loss(self):
        return self._loss


    def weight_variable(self, shape, name):
        """Create a weight variable with appropriate initialization."""
        initial = tf.truncated_normal(shape, stddev=0.1)
        return tf.Variable(initial, name=name)

    def bias_variable(self, shape, name):
        """Create a bias variable with appropriate initialization."""
        initial = tf.constant(0.1, shape=shape)
        return tf.Variable(initial, name=name)


        
    def dropout(self, x, keep_prob):
        return tf.nn.dropout(x, keep_prob)
        
    def gru_cell(self):
        with tf.name_scope('gru_cell'):
            cell = rnn.GRUCell(self.hidden_size, reuse=False)
        return rnn.DropoutWrapper(cell, output_keep_prob=self.keep_prob)

    def bi_gru(self, inputs):
        """build the bi-GRU network. 返回个所有层的隐含状态。"""
        cells_fw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        cells_bw = [self.gru_cell() for _ in range(self.n_layer_bigru)]
        initial_states_fw = [cell_fw.zero_state(self.batch_size, tf.float32) for cell_fw in cells_fw]
        initial_states_bw = [cell_bw.zero_state(self.batch_size, tf.float32) for cell_bw in cells_bw]
        outputs, _, _ = rnn.stack_bidirectional_dynamic_rnn(cells_fw, cells_bw, inputs,
                                                            initial_states_fw=initial_states_fw,
                                                            initial_states_bw=initial_states_bw, dtype=tf.float32)
        return outputs

    def task_specific_attention(self, inputs, output_size,
                                initializer=layers.xavier_initializer(),
                                activation_fn=tf.tanh, scope=None):
        assert len(inputs.get_shape()) == 3 and inputs.get_shape()[-1].value is not None
        with tf.variable_scope(scope or 'attention') as scope:
            # u_w, attention 向量
            attention_context_vector = tf.get_variable(name='attention_context_vector', shape=[output_size],
                                                       initializer=initializer, dtype=tf.float32)
            # 全连接层，把 h_i 转为 u_i ， shape= [batch_size, units, input_size] -> [batch_size, units, output_size]
            input_projection = layers.fully_connected(inputs, output_size, activation_fn=activation_fn, scope=scope)
            # 输出 [batch_size, units]
            vector_attn = tf.reduce_sum(tf.multiply(input_projection, attention_context_vector), axis=2, keep_dims=True)
            attention_weights = tf.nn.softmax(vector_attn, dim=1)
            tf.summary.histogram('attention_weigths', attention_weights)
            weighted_projection = tf.multiply(inputs, attention_weights)
            outputs = tf.reduce_sum(weighted_projection, axis=1)
            return outputs  # 输出 [batch_size, hidden_size*2]


    def bigru_inference(self, X_inputs):
        inputs=X_inputs
        output_bigru = self.bi_gru(inputs)
        output_att = self.task_specific_attention(output_bigru, self.hidden_size*2)
        return output_att

    def bn_layer(self,x, scope, is_training, epsilon=0.001, decay=0.99, reuse=None):
        with tf.variable_scope(scope, reuse=reuse):
            shape = x.get_shape().as_list()
            gamma = tf.get_variable(scope+"_gamma", shape[-1], initializer=tf.constant_initializer(1.0), trainable=True)
            beta = tf.get_variable(scope+"_beta", shape[-1], initializer=tf.constant_initializer(0.0), trainable=True)
            moving_avg = tf.get_variable(scope+"_moving_mean", shape[-1], initializer=tf.constant_initializer(0.0), trainable=False)
            moving_var = tf.get_variable(scope+"_moving_variance", shape[-1], initializer=tf.constant_initializer(1.0), trainable=False)
            if is_training is not None:
                avg, var = tf.nn.moments(x, np.arange(len(shape)-1), keep_dims=True)
                avg=tf.reshape(avg, [avg.shape.as_list()[-1]])
                var=tf.reshape(var, [var.shape.as_list()[-1]])
                update_moving_avg=tf.assign(moving_avg, moving_avg*decay+avg*(1-decay))
                update_moving_var=tf.assign(moving_var, moving_var*decay+var*(1-decay))
                control_inputs = [update_moving_avg, update_moving_var]
            else:
                avg = moving_avg
                var = moving_var
                control_inputs = []
            with tf.control_dependencies(control_inputs):
                output = tf.nn.batch_normalization(x, avg, var, offset=beta, scale=gamma, variance_epsilon=epsilon)
        return output

    def bn_layer_top(self,x, scope, is_training, epsilon=0.001, decay=0.99):
        return tf.cond(
            is_training,
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=True, reuse=None),
            lambda: self.bn_layer(x=x, scope=scope, epsilon=epsilon, decay=decay, is_training=False, reuse=True),
        )




def loader(data_store_path=''):############
    # all_data=pd.read_csv(data_store_path+"data_sets/train_feature_second_no_query_and_title_data.csv")
    # query_title=pd.read_csv(data_store_path+"data_sets/train_data.csv",usecols=['query','title'])

    print("开始加载数据，有ctr")
    # train_data=pd.read_pickle(data_store_path+"train_feature_first_no_query_and_title_data.pickle")[train_start_index:train_end_index].reset_index(drop=True)
    query_title=pd.read_pickle("/home/kesci/blending/train_data5000w-6000w.pickle")
    
    query_title=pd.concat([query_title,pd.DataFrame(np.zeros([10000000,1]))],axis=1)
    
    print("加载数据完成")
    return query_title,1

def batch_iter(data_x,data_y, batch_size, num_epochs, shuffle=False):#这个就是产生batch的，可以直接用
    """
    Generates a batch iterator for a dataset.
    """
    # data_x=data_x.tolist()
    data=list(zip(data_x,data_y))
    data=np.array(data)
    data_size = len(data)
    num_batches_per_epoch = int((len(data)-1)/batch_size) + 1
    for epoch in range(num_epochs):
        # Shuffle the data at each epoch
        if shuffle:#如果为true 则代表允许随机取数据
            shuffle_indices = np.random.permutation(np.arange(data_size))
            shuffled_data = data[shuffle_indices]#随机取数据
        else:
            shuffled_data = data
        for batch_num in range(num_batches_per_epoch):
            start_index = batch_num * batch_size
            end_index = min((batch_num + 1) * batch_size, data_size)
            yield shuffled_data[start_index:end_index]#而不用一个个加到列表里了。这是一个batch，嗯哼
            #取到的维度为（batchsize,2） y为(batchsize,1) 里面存的是序号

from gensim import models
import pickle
fw=open('dictionary/dictionary.pickle','rb')
dictionary=pickle.load(fw)
import pickle
fw=open('tfidf_model/tfidf_model.pickle','rb')
tfidf_model=pickle.load(fw)

def get_w2v_array(word_list,max_len):
    single_sentence_tfidf=tfidf_model[dictionary.doc2bow(word_list)]
    # single_sentence_tfidf={dictionary[word_id_tfidf[0]]:word_id_tfidf[1] for word_id_tfidf in single_sentence_tfidf}
    array = np.zeros((max_len, 100))
    if len(word_list)<=max_len:
        for i in range(len(word_list)):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]

                # except:
                #     pass
                
    else:
        for i in range(max_len):
            if str(word_list[i]) in word_wv.vocab.keys():
                array[i][:] = word_wv[str(word_list[i])]
                # try:
                #     array[i][:]*=single_sentence_tfidf[str(word_list[i])]
                # except:
                #     pass
    return array

settings = Settings()

train_batches,vaild_data=loader()

train_batches=np.array(train_batches)
train_batches_y=train_batches[:,2]
train_batches_x=np.delete(train_batches, 2, axis=1)######这里是除了label以外的维数


batches=batch_iter(train_batches_x,train_batches_y,settings.train_batch_size,1)


print("train starting")
w2v_model = Word2Vec.load('w2v_model3/w2v_all_data_model.txt')
word_wv= w2v_model.wv


settings = Settings()

settings.all_features_num=train_batches_x.shape[1]-2###########自动设置数值型特征数量
model = TEXTCNN( settings)

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    learning_rate = tf.train.exponential_decay(settings.lr, model.global_step, settings.decay_step,
                                        settings.decay_rate, staircase=True)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    grads_and_vars=optimizer.compute_gradients(model.loss)
    train_op=optimizer.apply_gradients(grads_and_vars,global_step=model.global_step)

config = tf.ConfigProto() 
config.gpu_options.allow_growth = True 
sess = tf.Session(config=config) #启动创建的模型
var_list = tf.trainable_variables()
g_list = tf.global_variables()
bn_moving_vars = [g for g in g_list if 'moving_mean' in g.name]
bn_moving_vars += [g for g in g_list if 'moving_variance' in g.name]
global_step_var=[g for g in g_list if 'Global_Step' in g.name]
var_list += bn_moving_vars
var_list += global_step_var
saver = tf.train.Saver(var_list)
sess.run(tf.global_variables_initializer())
saver.restore(sess, tf.train.latest_checkpoint('/home/kesci/work/first_zzp/tf_model_bigru/'))

per_train_loss_avg_list=[]
per_vaild_loss_avg_list=[]
print("start")


parts_time=[]
from time import time
result_list=[]
for step,batch in enumerate(batches):
    time1=time()
    batch_x,batch_y=zip(*batch)
    batch_y=np.array(batch_y)[:,None]
    batch_x=[i.tolist() for i in batch_x]
    batch_x=pd.DataFrame(batch_x)
    batch_x[[0,1]]=batch_x[[0,1]].applymap(lambda x:x.split(' '))
    batch_x=np.array(batch_x)
    #####[None,title_len,embedding_size]
    #np.apply_along_axis(lambda x:[5,5,5,5,5] if x[0]==2 else x,axis=2,arr=a)
    #query   title   title_count         title_ctr      label
    batch_x_query=np.apply_along_axis(lambda x:get_w2v_array(x[0],settings.query_len),axis=1,arr=batch_x)
    batch_x_title=np.apply_along_axis(lambda x:get_w2v_array(x[1],settings.title_len),axis=1,arr=batch_x)
    batch_x_ctr_fea=batch_x[:,2:]
    
    #     train_op=optimizer.apply_gradients(grads_and_vars,global_step=global_step)                              
    feed_dict = {model.embedded_x_query: batch_x_query,
                 model.embedded_x_title: batch_x_title,
                 model.batch_size: batch_x.shape[0], 
                 model.is_training: False, 
                 model.keep_prob: 1,}

    result=sess.run([model._y_pred],feed_dict)
    result_list.append(result)
    time2=time()
    parts_time.append(time2-time1)
    if step%10==0:
        print(result[0][0])  
    if step%100==0:  
        time_single_part=sum(parts_time)
        ed_time=(20000000/settings.train_batch_size-step)/100*time_single_part
        print("距预测完一整轮剩余时间：",ed_time/60," 分钟")
        parts_time=[]



In [2]:
result_list2=[pd.DataFrame(i[0]) for i in result_list]
y_pred=pd.concat(result_list2).reset_index(drop=True)
y_pred

In [3]:
y_pred.mean()

In [4]:
y_pred.to_pickle("/home/kesci/blending/textcnn_rnn_fea_chunwenben.pickle")